## Required Installation

In [1]:
import random
from tqdm import tqdm
from spacy.training import Example
!pip install pyspark
!pip install spacy-lookups-data
!python -m spacy download en_core_web_md

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488512 sha256=74a034435d66fa7c3f50e438fa22a0a55d495d6848e4b5d20cf73f15a2eb5e7f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 12.0 MB/s eta 0:00:0000:0100:01
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-

## PySpark Session and Data Setup

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType
import re

# Initialize a Spark session
spark = SparkSession.builder.appName("LogAnalysis").getOrCreate()
#spark = SparkSession.builder.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer").getOrCreate()

# Define the schema for log entries
schema = StructType([
    StructField("Timestamp", StringType(), True),
    StructField("Log Level", StringType(), True),
    StructField("Component", StringType(), True),
    StructField("Operation Type", StringType(), True),
    StructField("Block ID", StringType(), True),
    StructField("Source Address", StringType(), True),
    StructField("Destination Address", StringType(), True)
])

# Regular expression pattern to extract information
pattern = r'(\d{6} \d{6}) (\d+) (\w+) (\S+): (\w+) block (\S+) src: (\S+) dest: (\S+)'

# List to store extracted log entries
extracted_data = []

# Read and process the log file
with open('/kaggle/input/loghub-hadoop-distributed-file-system-log-data/HDFS_1/HDFS.log', 'r') as log_file:
    for line in log_file:
        match = re.match(pattern, line)
        if match:
            timestamp, log_level, component, _, operation_type, block_id, source_addr, dest_addr = match.groups()
            log_entry = (timestamp, log_level, component, operation_type, block_id, source_addr, dest_addr)
            extracted_data.append(log_entry)

# Create a DataFrame using the defined schema
df = spark.createDataFrame(extracted_data, schema=schema)

# Convert the "Timestamp" column to date format
df = df.withColumn("Timestamp", to_date(df["Timestamp"], "yyMMdd HHmmss"))

print("df Schema:")
df.printSchema()

# Extract the block number from Block ID using a regular expression
block_pattern = r'blk_-(\d+)'
df = df.withColumn("Block", regexp_extract("Block ID", block_pattern, 1))

# Read the "anomaly_label.csv" file
anomaly_label_df = spark.read.csv('/kaggle/input/loghub-hadoop-distributed-file-system-log-data/HDFS_1/anomaly_label.csv', header=True, inferSchema=True)
print("anomaly_label_df Schema:")
anomaly_label_df.printSchema()

# Join the DataFrames on BlockId
merged_df = df.join(anomaly_label_df, df["Block ID"] == anomaly_label_df["BlockId"], "left")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 17:41:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


df Schema:
root
 |-- Timestamp: date (nullable = true)
 |-- Log Level: string (nullable = true)
 |-- Component: string (nullable = true)
 |-- Operation Type: string (nullable = true)
 |-- Block ID: string (nullable = true)
 |-- Source Address: string (nullable = true)
 |-- Destination Address: string (nullable = true)



anomaly_label_df Schema:
root
 |-- BlockId: string (nullable = true)
 |-- Label: string (nullable = true)



In [3]:
merged_df.head(5)

24/04/06 17:42:37 WARN TaskSetManager: Stage 2 contains a task of very large size (49471 KiB). The maximum recommended task size is 1000 KiB.
24/04/06 17:42:43 WARN PythonRunner: Detected deadlock while completing task 3.0 in stage 2 (TID 8): Attempting to kill Python Worker
24/04/06 17:42:43 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 2 (TID 5): Attempting to kill Python Worker
24/04/06 17:42:43 WARN PythonRunner: Detected deadlock while completing task 2.0 in stage 2 (TID 7): Attempting to kill Python Worker
24/04/06 17:42:43 WARN PythonRunner: Detected deadlock while completing task 1.0 in stage 2 (TID 6): Attempting to kill Python Worker


[Row(Timestamp=datetime.date(2008, 11, 9), Log Level='143', Component='INFO', Operation Type='Receiving', Block ID='blk_-1608999687919862906', Source Address='/10.250.19.102:54106', Destination Address='/10.250.19.102:50010', Block='1608999687919862906', BlockId='blk_-1608999687919862906', Label='Normal'),
 Row(Timestamp=datetime.date(2008, 11, 9), Log Level='143', Component='INFO', Operation Type='Receiving', Block ID='blk_-1608999687919862906', Source Address='/10.250.10.6:40524', Destination Address='/10.250.10.6:50010', Block='1608999687919862906', BlockId='blk_-1608999687919862906', Label='Normal'),
 Row(Timestamp=datetime.date(2008, 11, 9), Log Level='145', Component='INFO', Operation Type='Receiving', Block ID='blk_-1608999687919862906', Source Address='/10.250.14.224:42420', Destination Address='/10.250.14.224:50010', Block='1608999687919862906', BlockId='blk_-1608999687919862906', Label='Normal'),
 Row(Timestamp=datetime.date(2008, 11, 10), Log Level='10371', Component='INFO',

In [4]:
merged_df.columns

['Timestamp',
 'Log Level',
 'Component',
 'Operation Type',
 'Block ID',
 'Source Address',
 'Destination Address',
 'Block',
 'BlockId',
 'Label']

# Cleaning The Dataset

In [5]:
# Drop duplicate entries based on BlockId
deduplicated_df = merged_df.dropDuplicates(["BlockId"])

# Select specific columns to create a new DataFrame
cleaned_df = deduplicated_df.select("Label", "BlockId", "Timestamp", "Log Level", "Operation Type")

# Show the new DataFrame
cleaned_df.show()

24/04/06 17:42:47 WARN TaskSetManager: Stage 7 contains a task of very large size (49471 KiB). The maximum recommended task size is 1000 KiB.


+-------+--------------------+----------+---------+--------------+
|  Label|             BlockId| Timestamp|Log Level|Operation Type|
+-------+--------------------+----------+---------+--------------+
| Normal|blk_-100001458415...|2008-11-10|     5887|     Receiving|
| Normal|blk_-100004553717...|2008-11-11|    19873|     Receiving|
| Normal|blk_-100021874837...|2008-11-10|    12321|     Receiving|
| Normal|blk_-100028489120...|2008-11-10|    11134|     Receiving|
| Normal|blk_-100029668880...|2008-11-10|    12376|     Receiving|
| Normal|blk_-100032145436...|2008-11-10|    12401|     Receiving|
| Normal|blk_-100041069666...|2008-11-11|    17873|     Receiving|
| Normal|blk_-100058394360...|2008-11-11|    19134|     Receiving|
| Normal|blk_-100080480388...|2008-11-11|    21852|     Receiving|
| Normal|blk_-100088503271...|2008-11-11|    18893|     Receiving|
| Normal|blk_-100092734476...|2008-11-09|     1385|     Receiving|
| Normal|blk_-100097373580...|2008-11-10|    11195|     Receiv

In [6]:
count = cleaned_df.select("BlockId").distinct().count()
print("Number of unique BlockIds:", count)

24/04/06 17:43:05 WARN TaskSetManager: Stage 16 contains a task of very large size (49471 KiB). The maximum recommended task size is 1000 KiB.


Number of unique BlockIds: 575061


In [7]:
cleaned_df.columns

['Label', 'BlockId', 'Timestamp', 'Log Level', 'Operation Type']

## Taking Only Sample For Training

- As we have total 575061 rows but it will be very hard and time consuming for now to train it. So for the purpose of coding challenge i will reduce the dataset

In [9]:
sampled_df = cleaned_df.sample(fraction=0.01)

In [10]:
sampled_df.count()

24/04/06 17:43:16 WARN TaskSetManager: Stage 30 contains a task of very large size (49471 KiB). The maximum recommended task size is 1000 KiB.


5861

In [11]:
sampled_df.columns

['Label', 'BlockId', 'Timestamp', 'Log Level', 'Operation Type']

In [12]:
sampled_df.describe().show()

24/04/06 17:43:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/04/06 17:43:26 WARN TaskSetManager: Stage 44 contains a task of very large size (49471 KiB). The maximum recommended task size is 1000 KiB.


+-------+-------+--------------------+------------------+--------------+
|summary|  Label|             BlockId|         Log Level|Operation Type|
+-------+-------+--------------------+------------------+--------------+
|  count|   5865|                5865|              5865|          5865|
|   mean|   NULL|                NULL|15701.821994884911|          NULL|
| stddev|   NULL|                NULL|  7656.92725275512|          NULL|
|    min|Anomaly|blk_-100127896875...|              1000|     Receiving|
|    max| Normal|blk_9965052949101...|              9952|     Receiving|
+-------+-------+--------------------+------------------+--------------+



## Prepare Data

In [13]:
import pandas as pd
import spacy
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

data = sampled_df.toPandas()

# Define the entities and patterns to anonymize
entities = ["BlockId"]
patterns = [{"label": "KEYWORD", "pattern": [{"LOWER": "receiving"}]}]

# Prepare data for spaCy
def prepare_data(data):
    train_data = []
    for index, row in data.iterrows():
        text = f"{row['Label']} {row['BlockId']} {row['Timestamp']} {row['Log Level']} {row['Operation Type']}"
        ents = [(row['BlockId'].start(), row['BlockId'].end(), "BlockId")]
        train_data.append((text, {"entities": ents}))
    return train_data

24/04/06 17:43:40 WARN TaskSetManager: Stage 58 contains a task of very large size (49471 KiB). The maximum recommended task size is 1000 KiB.


In [14]:
import re
def prepare_data_(data):
    train_data = []
    block_id_pattern = re.compile("blk_[-\d]+")  # Adjust the pattern as needed
    for index, row in data.iterrows():
        text = f"{row['Label']} {row['BlockId']} {row['Timestamp']} {row['Log Level']} {row['Operation Type']}"
        match = block_id_pattern.search(text)
        if match:
            start, end = match.span()
            ents = [(start, end, "BlockId")]
            train_data.append((text, {"entities": ents}))
    return train_data

## Splitting Data

In [15]:
# Split data into train and test sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [16]:
# Prepare training and testing data
train_data = prepare_data_(train_df)
test_data = prepare_data_(test_df)

## Training Model

In [17]:
# Load the spaCy model
nlp = spacy.load("en_core_web_md")

In [18]:
# Add the entity ruler to the pipeline
ruler = nlp.add_pipe("entity_ruler", before="ner")
ruler.add_patterns(patterns)

In [19]:
# Convert the training data to DocBin format
db = DocBin()
for text, annotations in train_data:
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annotations["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is not None:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

In [20]:
# Save the DocBin object
db.to_disk("./train.spacy")

In [30]:
# Train the NER model
ner = nlp.get_pipe("ner")
optimizer = nlp.create_optimizer()

In [44]:
# Disable other pipeline components during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    # Calculate total number of training steps
    total_steps = len(train_data)
    
    # Use tqdm to create a progress bar
    with tqdm(total=total_steps, desc="Training NER") as pbar:
        for itn in range(5):
            print("count", itn)
            random.shuffle(train_data)
            losses = {}
            for text, annotations in train_data:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                ner.update([example], drop=0.5, losses=losses)
                pbar.update(1)  # Update progress bar after each step
            print(losses)

Training NER:   0%|          | 7/4686 [00:00<01:09, 67.63it/s]

count 0


Training NER: 4698it [01:01, 75.92it/s]                          

{'ner': 7.658487340958925e-05}
count 1


Training NER: 9387it [02:02, 75.80it/s]

...

{'ner': 0.009784093102804883}
count 3


Training NER: 18752it [04:04, 75.28it/s]

{'ner': 0.32134077670114386}
count 4


Training NER: 23430it [05:05, 76.68it/s]

{'ner': 0.7968169886826436}


## Saving Model

In [45]:
# Save the trained model
nlp.to_disk("./main_ner_model")

In [46]:
# Test the model
test_texts = [f"{row['Label']} {row['BlockId']} {row['Timestamp']} {row['Log Level']} {row['Operation Type']}" for index, row in test_df.iterrows()]
docs = list(nlp.pipe(test_texts))

In [47]:
len(docs)

1172

In [39]:
# # Print entities for manual inspection
# for doc in docs:
#     for ent in doc.ents:
#         print(ent.text, ent.label_)

## Testing Model

In [48]:
def anonymize_block_ids(text):
    doc = nlp(text)
    anonymized_text = []
    last_idx = 0
    for ent in doc.ents:
        if ent.label_ == "BlockId":
            # Anonymize BlockId with a generic string
            anonymized_text.append(text[last_idx:ent.start_char])
            anonymized_text.append("[BLOCK_ID]")
            last_idx = ent.end_char
    anonymized_text.append(text[last_idx:])
    return "".join(anonymized_text)

# Test Example
test_text = "Normal blk_-100001458415... 2008-11-10 5887 Receiving"
anonymized_text = anonymize_block_ids(test_text)
print("Original Text:", test_text)
print("Anonymized Text:", anonymized_text)

Original Text: Normal blk_-100001458415... 2008-11-10 5887 Receiving
Anonymized Text: Normal [BLOCK_ID][BLOCK_ID] 2008-11-10 5887 Receiving


In [53]:
# This will be our lookup table (in a real project, this would need secure storage and management)
lookup_table = {
    "[BLOCK_ID_1]": "blk_-100001458415...",
    "[BLOCK_ID_2]": "blk_-100021874837...",
    # ... more entries ...
}

def retrieve_block_ids(text):
    doc = nlp(text)
    retrieved_text = []
    last_idx = 0
    for ent in doc.ents:
        if ent.label_ == "BlockId":
            anonymized_id = text[ent.start_char:ent.end_char]
            original_id = lookup_table.get(anonymized_id)  # It will retrieve from lookup table
            if original_id:
                retrieved_text.append(text[last_idx:ent.start_char])
                retrieved_text.append(original_id)
                last_idx = ent.end_char
            else:
                # Handle cases where the anonymized ID is not found
                retrieved_text.append(text[last_idx:ent.end_char])
                last_idx = ent.end_char
    retrieved_text.append(text[last_idx:])
    return "".join(retrieved_text)

# Test Example
anonymized_text = "Normal [BLOCK_ID_1] 2008-11-10 5887 Receiving"
retrieved_text = retrieve_block_ids(anonymized_text)
print("Anonymized Text:", anonymized_text)
print("Retrieved Text:", retrieved_text)

Anonymized Text: Normal [BLOCK_ID_1] 2008-11-10 5887 Receiving
Retrieved Text: Normal blk_-100001458415... 2008-11-10 5887 Receiving
